In [ ]:
!apt-get update
!apt-get install -y build-essential libzbar-dev
!apt install python3-zbar

In [2]:
!pip install -q streamlit
!pip install pyzbar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [3]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.85s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [4]:
!git clone https://github.com/jaysheel-ops/barcode-detection-repo

Cloning into 'barcode-detection-repo'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
Receiving objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0


In [5]:
!mv barcode-detection-repo/* /content/

In [6]:
%%writefile app.py

import streamlit as st
from PIL import Image
from pyzbar.pyzbar import decode
import json
from pathlib import Path


file_path = Path("cart.json")
if not file_path.exists():
    with open('cart.json', 'w') as f1:
        f1.write("{\n}")

# Define your custom function
def run_scanner(image):
    # Your custom logic here
    # For demonstration purposes, let's just convert the image to grayscale
    data = decode(image)[0][0]
    return data

# Pre-defined dictionary
item_dictionary = {
    "02345673": {'name': 'banana', 'price': '10'},
    "02345147": {'name': 'apple', 'price': '30'},
    "06397126": {'name': 'mango', 'price': '40'},
    "08679339": {'name': 'pie', 'price': '50'}
}


def process_upc_e_binary(binary_input):
    # Decode binary to string
    decoded_string = binary_input.decode('utf-8')
    decoded_string = decoded_string.replace("0000", "")
    decoded_string = decoded_string.replace("00000", "")
    decoded_string = decoded_string.replace("00", "0")

    return decoded_string

def check_updated(uploaded_file, cart):
    if uploaded_file.type.startswith('image/'):
        st.image(uploaded_file, caption="Uploaded Image.", use_column_width=True)

        # Run the custom function
        processed_image = run_scanner(Image.open(uploaded_file))

        processed_data = process_upc_e_binary(processed_image)

        if processed_data not in cart:
            cart[processed_data] = 1
        else:
            cart[processed_data] += 1

        json_object = json.dumps(cart, indent=4)
        with open("cart.json", "r+") as outfile:
            outfile.write(json_object)

        st.write("Items in cart (Name - Quantity)")
        for key, value in cart.items():
            if key in item_dictionary:
                value = item_dictionary[key]
                st.write(f"{value['name']} - {cart[key]}")

# Streamlit app
def main():
    st.title("Image Processing and Shopping Cart App")
    cart = json.load(open('cart.json', "r+"))

    # Upload image through Streamlit
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])
    if uploaded_file:
        # Check if the format is .jpg or .png
        cart = check_updated(uploaded_file, cart)
    else:
        st.write("Items in cart (Name - Quantity)")
        for key, value in cart.items():
            if key in item_dictionary:
                value = item_dictionary[key]
                st.write(f"{value['name']} - {cart[key]}")

main()

Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

34.73.4.80
npx: installed 22 in 3.37s
your url is: https://loose-lights-brush.loca.lt
